긍부정단어 출력하기

In [17]:
from google.colab import files
myfile = files.upload()

Saving 에타리뷰_수정.csv to 에타리뷰_수정.csv


In [18]:
import pandas as pd

df = pd.read_csv('에타리뷰_수정.csv')

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# 불용어사전
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, []) #2차원 리스트를 1차원 리스트로

불용어 = ['교수','님','건대','영화','고','으며','나','게','면','수','은','는','이','가'] # 나만의 불용어 추가

stopwords += 불용어 #합치기

In [21]:
제거품사 = ['IC','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC',
        'XPN','XSN','XSV','XSA','XR','SF','SE','SS','SP','SO','SW']
# 제거품사 = 감탄사. 조사, 접두사/접미사, 부호
# 추가 : 연결어미 (EC)

In [22]:
# 토크나이징 함수 (토크나이징 후 불용어까지 제거)

from konlpy.tag import Mecab
def tokenizing(data):
  m = Mecab()
  words = m.pos(data)
  words = [word for word,tag in words if tag not in 제거품사 and word not in stopwords]
  return words

In [23]:
# 전처리 함수
import re

def cleaning(text):
    hangul = re.compile('[^ 가-힣]')  # 한글 추출(ㅋㅋㅋ 등 제외)
    result = hangul.sub('', text)  # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    result = re.sub(' +',' ', result) #띄어쓰기 2개 이상이면 1개로
    return result

In [24]:
for i in range(len(df)): #리뷰데이터 전처리
  df['리뷰'][i] = cleaning(df['리뷰'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
# 긍정 4점이상, 부정 1,2점만 있는 테이블 만들기

긍정 = df[df['평점']>3]
부정 = df[df['평점'] < 3]
result = pd.concat([긍정,부정])
result.reset_index(inplace=True)

In [26]:
# 긍정에 1, 부정에 0 
def rating_to_label(rating):
    if rating > 3:
        return 1 #긍정
    else:
        return 0 #부정
    
result['y'] = result['평점'].apply(lambda x: rating_to_label(x))

In [27]:
# result 의 리뷰 전체 코퍼스
리뷰전체 = []
for i in range(len(result)):
  리뷰전체.append(result['리뷰'][i])

In [28]:
#bi-gram & 벡터화
vectorizer = CountVectorizer(tokenizer = tokenizing ,ngram_range=(2,2))
X1 = vectorizer.fit_transform(리뷰전체) 
features = (vectorizer.get_feature_names())
print(features)
print("\n\nX1 : \n", X1.toarray())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['가간 엔', '가감 방식', '가관 임', '가까운 같', '가까운 공공', '가까운 광고', '가까운 내용', '가까운 단계', '가까운 데', '가까운 듯', '가까운 분', '가까운 영양소', '가까운 점수', '가까운 학생', '가까울 정도', '가까움 본인', '가까움 음악사', '가까움 주제', '가까움 토론', '가까워 교양', '가까워 서', '가까워 요', '가까워 지', '가까워 지려', '가까워서 재밌', '가까워야 받', '가까워질 임', '가까워질수록 말', '가까이 나왔', '가까이 다가갈', '가까이 되', '가까이 됨', '가까이 듣', '가까이 등록금', '가까이 맞', '가까이 받', '가까이 앉', '가까이 알', '가까이 일찍', '가까이 잘', '가까이 했', '가까이 화면', '가깝 기', '가깝 다', '가깝 다고', '가깝 다는', '가깝 받', '가깝 역사', '가깝 자세히', '가깝 재밌', '가깝 점', '가깝 지', '가깝 흥미', '가꾸 데', '가끔 개그', '가끔 교과서', '가끔 귀여우', '가끔 까다로운', '가끔 끼어드', '가끔 나오', '가끔 내용', '가끔 내주', '가끔 너무', '가끔 농담', '가끔 대답', '가끔 독후감', '가끔 듣', '가끔 라도', '가끔 말', '가끔 문제', '가끔 미술', '가끔 발표', '가끔 보', '가끔 보여', '가끔 부르', '가끔 북한', '가끔 분', '가끔 비유', '가끔 빈자리', '가끔 빨리', '가끔 사탕', '가끔 사회', '가끔 산', '가끔 살짝', '가끔 생각', '가끔 설명', '가끔 수업', '가끔 쉬', '가끔 시', '가끔 시험', '가끔 실시간', '가끔 실험실', '가끔 아무', '가끔 안', '가끔 앞뒤', '가끔 어동', '가끔 언급', '가끔 엄청', '가끔 에타', '가끔 여러', '가끔 열린', '가끔 예습', '가끔 울먹울먹', '가끔 웃겨요', '가끔 은근', '가끔 이게', '가끔 이름', '가끔 이순신',

In [29]:
# 벡터화
X2 = vectorizer.fit_transform(리뷰전체)
scores = (X2.toarray())
print("\n\nScores : \n", scores)



Scores : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [30]:
# Getting top ranking features
sums = X2.sum(axis = 0) #컬럼별로 sum
data1 = []

for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))

ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False)) #rank 큰 순서대로 (내림차순)
print ("\n\nWords : \n", words)



Words : 
           term  rank
105884     지 않  2293
103124     주 시  2238
122448     해 주  1571
102685     좋 았  1166
119991     한 번   924
...        ...   ...
48539   미국 뮤지션     1
48537     미국 맞     1
48536     미국 라     1
48533    미국 대학     1
126199   힙합 장르     1

[126200 rows x 2 columns]


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    tokenizer = tokenizing,
    ngram_range=(2,2)
)
train_x = vectorizer.fit_transform(리뷰전체)
train_x.shape

(9963, 126200)

In [32]:
train_label = result['y']

In [33]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_x, train_label)

idxs_coef = list(enumerate(classifier.coef_[0]))
positive_idxs = sorted(idxs_coef, key=lambda x:-x[1])[:150]
negative_idxs = sorted(idxs_coef, key=lambda x:x[1])[:150] #긍부정 단어 150개

vocab2idx = vectorizer.vocabulary_
idx2vocab = list(sorted(vocab2idx, key=lambda x:vocab2idx[x]))

In [34]:
neg = []
for index, 수치 in negative_idxs:
  for key, value in vocab2idx.items():
    if index == value:
      neg.append(key)

In [35]:
pos = []
for index, 수치 in positive_idxs:
  for key, value in vocab2idx.items():
    if index == value:
      pos.append(key)

In [43]:
# 최종 긍정단어
pos = ['어렵 지',
 '재밌 었',
 '너무 좋',
 '부담 없',
 '정말 좋',
 '좋 았',
 '얻 많',
 '친절 시',
 '내용 재밌',
 '감사 합니다',
 '재밌 음',
 '내용 좋',
 '좋 수업',
 '잘 나오',
 '개꿀 강의',
 '주 셔서',
 '많 아서',
 '사랑 합니다',
 '될 듯',
 '수업 잘',
 '좋 으세요',
 '거 많',
 '한 번',
 '수업 재밌',
 '강의 재밌',
 '주 시',
 '문제 없',
 '신청 성공',
 '꼭 으세요',
 '좋 음',
 '꿀 강의',
 '교양 다운',
 '다 알려',
 '기준 잘',
 '받 있',
 '잘 주심',
 '교양 강의',
 '잘 해',
 '있 좋',
 '쉬운 편',
 '유익 했',
 '무조건 으세요',
 '수업 재미있',
 '라고 생각',
 '괜찮 았',
 '주 십니다',
 '갈 있',
 '진짜 좋',
 '시험 준비',
 '에이플 받',
 '에이쁠 받',
 '잘 었',
 '재미있 었',
 '힘들 지',
 '재밌 수업',
 '정말 재밌',
 '쉬 움',
 '과제 매주',
 '좋 아요',
 '관심 있',
 '너무 재밌',
 '있 어서',
 '강의 다',
 '번 보',
 '과제 한',
 '알려 주심',
 '많 긴',
 '에이쁠 가능',
 '흥미 로웠',
 '한 만큼',
 '시험 쉽',
 '좋 강의',
 '최고 강의',
 '해 주심',
 '편하 있',
 '관심 많',
 '개꿀 과목',
 '많 아요',
 '다 재미없',
 '과제 적',
 '부담없이 있',
 '도움 많이',
 '가능 합니다',
 '좋 으시',
 '채우 기',
 '기 쉽',
 '다 듣',
 '필기 잘',
 '최고 교양',
 '안 해도',
 '할 만',
 '좋 으십니다',
 '중간고사 대체',
 '어서 좋',
 '학점 느',
 '잘 들으면',
 '범위 많',
 '과제 귀찮',
 '만족 함',
 '무조건 들어야',
 '좋 으셔서',
 '친절 심',
 '좋 같',
 '개꿀 교양',
 '강의 학점',
 '달달 외우',
 '시험 피피티',
 '강의 좋',
 '부담 적']

In [44]:
neg = ['듣 지',
 '별로 였',
 '없 다',
 '재미 없',
 '듣 겠',
 '없 어요',
 '얻 없',
 '지 마세요',
 '없 음',
 '거 없',
 '점 줌',
 '잘 안',
 '지 마',
 '배운 있',
 '진짜 별로',
 '정말 별로',
 '안 됨',
 '그대로 읽',
 '별로 임',
 '스트레스 받',
 '강의 너무',
 '모르 겠',
 '지 못하',
 '안 맞',
 '도망쳐 빨리',
 '안 좋',
 '너무 별로',
 '그랬 어요',
 '짜 주',
 '진짜 함',
 '후회 강의',
 '너무 함',
 '많 시험',
 '안 듣',
 '졸리 다',
 '안 줌',
 '별로 인',
 '신청 했',
 '있 그냥',
 '수업 노잼',
 '학점 짜',
 '좋 겠',
 '안 알려',
 '느낌 과제',
 '문제 풀',
 '거 없이',
 '강의 최악',
 '어렵 배운',
 '는지 모르',
 '틀린 해석',
 '괜히 었',
 '과제 학점',
 '배워가 거',
 '늦 올림',
 '강의 피하',
 '기 싫',
 '올려 주',
 '소신 같',
 '정치 소신',
 '같 으면',
 '기 불편',
 '최악 강의',
 '줄 알',
 '시 긴',
 '어요 한',
 '안 채워',
 '거르 세요',
 '피하 시',
 '수업 재미',
 '합니다 시험',
 '맞 아',
 '채점 기준',
 '다 출석',
 '안 해',
 '수업 재미없',
 '초청 강사',
 '과제 출석',
 '너무 합니다',
 '수업 온라인',
 '안 붙여',
 '없 수업',
 '오래 설명',
 '수업 준비',
 '플 매주',
 '주 아님',
 '제일 별로',
 '책 읽',
 '재미없 었',
 '강의 평가',
 '아닌 같',
 '듯 학점',
 '없 강의',
 '아깝 다',
 '안 되',
 '내용 그닥',
 '그럭저럭 절',
 '었 는지',
 '주 싶',
 '싫 으면',
 '문제 중',
 '안 됬',
 '쉬운 데',
 '해야 합니다',
 '내용 어렵',
 '학생 발표',
 '열심히 으셔야',
 '과제 화났',
 '화났 어요',
 '제대로 점수',
 '내용 그럭저럭',
 '피하 십시오']

전체 데이터

In [38]:
# 전체 리뷰 코퍼스
corpus = []
for i in range(len(df)):
    corpus.append(df['리뷰'][i])

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(tokenizer = tokenizing ,ngram_range=(2,2)) #bi-gram & tf-idf
tfidfv.fit(corpus)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(ngram_range=(2, 2),
                tokenizer=<function tokenizing at 0x7fc399a37c20>)

In [40]:
DF = tfidfv.transform(corpus).toarray()

dataframe = pd.DataFrame(DF, columns = sorted(tfidfv.vocabulary_))

In [46]:
posDF = dataframe[pos]
posDF["posSum"] = posDF.sum(axis = 1) #긍정 단어
posDF

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,어렵 지,재밌 었,너무 좋,부담 없,정말 좋,좋 았,얻 많,친절 시,내용 재밌,감사 합니다,재밌 음,내용 좋,좋 수업,잘 나오,개꿀 강의,주 셔서,많 아서,사랑 합니다,될 듯,수업 잘,좋 으세요,거 많,한 번,수업 재밌,강의 재밌,주 시,문제 없,신청 성공,꼭 으세요,좋 음,꿀 강의,교양 다운,다 알려,기준 잘,받 있,잘 주심,교양 강의,잘 해,있 좋,쉬운 편,...,시험 쉽,좋 강의,최고 강의,해 주심,편하 있,관심 많,개꿀 과목,많 아요,다 재미없,과제 적,부담없이 있,도움 많이,가능 합니다,좋 으시,채우 기,기 쉽,다 듣,필기 잘,최고 교양,안 해도,할 만,좋 으십니다,중간고사 대체,어서 좋,학점 느,잘 들으면,범위 많,과제 귀찮,만족 함,무조건 들어야,좋 으셔서,친절 심,좋 같,개꿀 교양,강의 학점,달달 외우,시험 피피티,강의 좋,부담 적,posSum
0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.108449
1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10846,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.108460
2,0.000000,0.0,0.110974,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.12356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.234535
3,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.100939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.100939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11640,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.09458,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.094580
11641,0.000000,0.0,0.000000,0.000000,0.0,0.060770,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108787,0.0,0.0,0.0,0.0,0.0,0.0,0.083739,0.0,0.253295
11642,0.047193,0.0,0.000000,0.055628,0.0,0.041498,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.053087,0.0,0.0,0.0,0.0,0.0,0.043778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.241184
11643,0.000000,0.0,0.000000,0.000000,0.0,0.055016,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.070379,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [45]:
negDF = dataframe[neg]
negDF["negSum"] = negDF.sum(axis = 1) # 부정단어
negDF

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,듣 지,별로 였,없 다,재미 없,듣 겠,없 어요,얻 없,지 마세요,없 음,거 없,점 줌,잘 안,지 마,배운 있,진짜 별로,정말 별로,안 됨,그대로 읽,별로 임,스트레스 받,강의 너무,모르 겠,지 못하,안 맞,도망쳐 빨리,안 좋,너무 별로,그랬 어요,짜 주,진짜 함,후회 강의,너무 함,많 시험,안 듣,졸리 다,안 줌,별로 인,신청 했,있 그냥,수업 노잼,...,다 출석,안 해,수업 재미없,초청 강사,과제 출석,너무 합니다,수업 온라인,안 붙여,없 수업,오래 설명,수업 준비,플 매주,주 아님,제일 별로,책 읽,재미없 었,강의 평가,아닌 같,듯 학점,없 강의,아깝 다,안 되,내용 그닥,그럭저럭 절,었 는지,주 싶,싫 으면,문제 중,안 됬,쉬운 데,해야 합니다,내용 어렵,학생 발표,열심히 으셔야,과제 화났,화났 어요,제대로 점수,내용 그럭저럭,피하 십시오,negSum
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.081392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159367
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11640,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
11641,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
11642,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
11643,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [47]:
# df합치기(긍+부)

DATAFRAME = pd.concat([posDF['posSum'], negDF['negSum']], axis = 1)

In [49]:
slicingDF = df[['교과목명','교수명','성적부여','과제','팀플','시험횟수','평점']]

In [50]:
df2 = pd.concat([slicingDF, DATAFRAME],axis = 1)

In [51]:
df2['score'] = 0.0
for i in range(len(df2)): # 감성 점수 계산
  if (df2['posSum'][i] + df2['negSum'][i]) != 0:
    df2['score'][i] = ((df2['posSum'][i] - df2['negSum'][i]) / (df2['posSum'][i] + df2['negSum'][i]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
df2['newScore'] = df2['평점'] + df2['score'] # newScore = 평점 + 감성점수

In [53]:
final = df2.groupby(['교과목명','교수명']).mean()